In [1]:
import ee
import geemap
import pandas as pd
from datetime import datetime

# 1. Inicializar
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# 2. Definir fechas: Desde 2003 hasta hoy
start_date = '2003-01-01'
end_date = datetime.today().strftime('%Y-%m-%d') # Fecha actual automática

print(f"Solicitando datos desde {start_date} hasta {end_date}...")

# 3. Límites de Tailandia (Provincias)
thailand_provinces = ee.FeatureCollection("FAO/GAUL/2015/level1") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

# 4. Dataset ERA5-Land Mensual
dataset = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
    .filterDate(start_date, end_date) \
    .select(['temperature_2m', 'dewpoint_temperature_2m', 'total_precipitation_sum'])

# 5. Función de reducción (Promedio por provincia)
def get_stats(image):
    date = image.date().format('YYYY-MM-dd')
    stats = image.reduceRegions(
        collection=thailand_provinces,
        reducer=ee.Reducer.mean(),
        scale=9000
    )
    return stats.map(lambda f: f.set('date', date))

# 6. Procesar en la nube
flat_col = dataset.map(get_stats).flatten()

# 7. Descargar datos (Método robusto corregido)
# Primero bajamos los nombres de las columnas para evitar el error de len()
columns = flat_col.first().propertyNames().getInfo()
data_dict = flat_col.reduceColumns(
    ee.Reducer.toList(len(columns)),
    columns
).getInfo()

df_tha = pd.DataFrame(data_dict['list'], columns=columns)

# 8. Limpieza y Cálculos (Humedad, Celsius)
df_tha['temp_c'] = df_tha['temperature_2m'] - 273.15
df_tha['dew_c'] = df_tha['dewpoint_temperature_2m'] - 273.15

import numpy as np
def calc_rh(row):
    t = row['temp_c']
    td = row['dew_c']
    # Fórmula de August-Roche-Magnus
    return 100 * (np.exp((17.625 * td)/(243.04 + td)) / np.exp((17.625 * t)/(243.04 + t)))

df_tha['rh_mean'] = df_tha.apply(calc_rh, axis=1)

# Selección final
cols_finales = ['ADM1_NAME', 'date', 'temp_c', 'rh_mean', 'total_precipitation_sum']
df_tha_final = df_tha[cols_finales].sort_values(['ADM1_NAME', 'date'])

print("¡Descarga completa!")
print(df_tha_final.head())
print(f"Total de registros: {len(df_tha_final)}")

c:\Users\Nainho\miniconda3\envs\licitacion\Lib\site-packages\geemap\conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Solicitando datos desde 2003-01-01 hasta 2026-01-16...
¡Descarga completa!
         ADM1_NAME        date     temp_c    rh_mean  total_precipitation_sum
0    Amnat Charoen  2003-01-01  22.352057  58.438984                 0.000094
76   Amnat Charoen  2003-02-01  26.014932  60.891920                 0.016480
152  Amnat Charoen  2003-03-01  26.755043  63.808427                 0.060445
228  Amnat Charoen  2003-04-01  29.417105  65.021982                 0.074617
304  Amnat Charoen  2003-05-01  28.790411  71.718316                 0.242831
Total de registros: 20976


In [3]:
df_tha_final.to_csv("data\clima_thai.csv",index=False)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'


C:\Users\Nainho\AppData\Local\Temp\ipykernel_1504\3739954883.py:1: SyntaxWarning: invalid escape sequence '\c'
  df_tha_final.to_csv("data\clima_thai.csv",index=False)
